In [ ]:
# 추천시스템(오늘 연습문제)
# 예상평점 구해보자
#     - 예상 점수 : 유사도 * (상대방의) 평점
#     ex) '강감찬'이 안본 영화중 'A'영화를
#     다른 사람들은 각각 4, 3, 2점 평점 줬다고 가정.
#     유사도를 구해보니 0.9, 0.3, 0.7로 나왔다면,
#     0.9*4점 => 3.6점 예상
#     0.3*3점 => 0.9점 예상
#     0.7*2점 => 1.4점 예상
#     예상점수 총합 = 5.9점(예상점수합)/1.9점(유사도합)
#     =>약 3점 예상됨
#     ...
#     만약 A영화 3점, B영화 2점, C영화 4점으로 예상됐다면
#     강감찬에게 'C'영화를 추천

# 2. 캐글 영화(음악) 데이터에 위 추천시스템 적용

In [1]:
# day16_quizbyGHG
import numpy as np
import pandas as pd

In [2]:
critics = {
    '조용필': {
        '택시운전사': 2.5,
        '겨울왕국': 3.5,
        '리빙라스베가스': 3.0,
        '넘버3': 3.5,
        '사랑과전쟁': 2.5,
        '세계대전': 3.0,
    },
    'BTS': {
        '택시운전사': 1.0,
        '겨울왕국': 4.5,
        '리빙라스베가스': 0.5,
        '넘버3': 1.5,
        '사랑과전쟁': 4.5,
        '세계대전': 5.0,
    },
    '강감찬': {
        '택시운전사': 3.0,
        '겨울왕국': 3.5,
        '리빙라스베가스': 1.5,
        '넘버3': 5.0,
        '세계대전': 3.0,
        '사랑과전쟁': 3.5,
    },
    '을지문덕': {
        '택시운전사': 2.5,
        '겨울왕국': 3.0,
        '넘버3': 3.5,
        '세계대전': 4.0,
    },
    '김유신': {
        '겨울왕국': 3.5,
        '리빙라스베가스': 3.0,
        '세계대전': 4.5,
        '넘버3': 4.0,
        '사랑과전쟁': 2.5,
    },
    '유성룡': {
        '택시운전사': 3.0,
        '겨울왕국': 4.0,
        '리빙라스베가스': 2.0,
        '넘버3': 3.0,
        '세계대전': 3.5,
        '사랑과전쟁': 2.0,
    },
    '이황': {
        '택시운전사': 3.0,
        '겨울왕국': 4.0,
        '세계대전': 3.0,
        '넘버3': 5.0,
        '사랑과전쟁': 3.5,
    },
    '이이': {'겨울왕국': 4.5, '사랑과전쟁': 1.0,
             '넘버3': 4.0},
}

In [3]:
def sim_pearson(data, user1, user2):
  '''
  두 사람 사이의 상관계수 계산
  
  '''
  li1 = []
  li2 = []
  pearson_sim = None
    
  for movie_name in data[user1]:
    if movie_name in data[user2]:
      user1_movie_score = data[user1][movie_name]
      user2_movie_score = data[user2][movie_name]
      li1.append(user1_movie_score)
      li2.append(user2_movie_score)
  n = len(li1)
  
  li1 = np.array(li1)
  li2 = np.array(li2)
  
  X = li1 - np.mean(li1)
  S_X = np.std(li1)
 
  Y = li2 - np.mean(li2)
  S_Y = np.std(li2)
  
  block_dot = np.dot(X, Y.T) / (n)
  
  pearson_sim = block_dot / (S_X * S_Y)
  
  #print(block_dot, S_X, S_Y)
  
  return pearson_sim

sim_pearson(critics, 'BTS', '이황')


-0.2844627935584563

In [4]:
def corr_matrix(data):
  '''
  상관계수 구하기
  '''
  res_df = pd.DataFrame()
  
  corr_dict = {}  
  
  for target_user in data:
    sim_values = []
    
    for reference_user in data:      
      sim_values.append(sim_pearson(data, target_user, reference_user))
      
    corr_dict[target_user] = sim_values
  
  result = pd.DataFrame(corr_dict, index=corr_dict.keys())
  
  return result
  
corr_matrix(critics)

,조용필,BTS,강감찬,을지문덕,김유신,유성룡,이황,이이
조용필,1.000000,0.054772,0.396059,0.404520,0.566947,0.558291,0.747018,0.991241
BTS,0.054772,1.000000,0.216930,0.569210,0.154303,0.417911,-0.284463,-0.381246
강감찬,0.396059,0.216930,1.000000,0.204598,0.314970,0.359313,0.963796,0.381246
을지문덕,0.404520,0.569210,0.204598,1.000000,1.000000,0.134840,0.134840,-1.000000
김유신,0.566947,0.154303,0.314970,1.000000,1.000000,0.707107,0.028571,0.893405
유성룡,0.558291,0.417911,0.359313,0.134840,0.707107,1.000000,0.060437,0.924473
이황,0.747018,-0.284463,0.963796,0.134840,0.028571,0.060437,1.000000,0.662849
이이,0.991241,-0.381246,0.381246,-1.000000,0.893405,0.924473,0.662849,1.000000


In [5]:
df = pd.DataFrame(critics)
df = df.T
df_corr = corr_matrix(critics)

In [6]:
df # 평점

,겨울왕국,넘버3,리빙라스베가스,사랑과전쟁,세계대전,택시운전사
조용필,3.5,3.5,3.0,2.5,3.0,2.5
BTS,4.5,1.5,0.5,4.5,5.0,1.0
강감찬,3.5,5.0,1.5,3.5,3.0,3.0
을지문덕,3.0,3.5,NaN,NaN,4.0,2.5
김유신,3.5,4.0,3.0,2.5,4.5,NaN
유성룡,4.0,3.0,2.0,2.0,3.5,3.0
이황,4.0,5.0,NaN,3.5,3.0,3.0
이이,4.5,4.0,NaN,1.0,NaN,NaN


In [7]:
df_corr # 유사도

,조용필,BTS,강감찬,을지문덕,김유신,유성룡,이황,이이
조용필,1.000000,0.054772,0.396059,0.404520,0.566947,0.558291,0.747018,0.991241
BTS,0.054772,1.000000,0.216930,0.569210,0.154303,0.417911,-0.284463,-0.381246
강감찬,0.396059,0.216930,1.000000,0.204598,0.314970,0.359313,0.963796,0.381246
을지문덕,0.404520,0.569210,0.204598,1.000000,1.000000,0.134840,0.134840,-1.000000
김유신,0.566947,0.154303,0.314970,1.000000,1.000000,0.707107,0.028571,0.893405
유성룡,0.558291,0.417911,0.359313,0.134840,0.707107,1.000000,0.060437,0.924473
이황,0.747018,-0.284463,0.963796,0.134840,0.028571,0.060437,1.000000,0.662849
이이,0.991241,-0.381246,0.381246,-1.000000,0.893405,0.924473,0.662849,1.000000


In [8]:
def predict_score(user_name, movies, data_score, data_corr):
  
  # 예측할 영화를 본사람들이 준 평점
  predict_dict = {}
  
  for movie_name in movies:
    # 그 영화를 본사람들의 점수
    scores = data_score[data_score[movie_name].isnull() == False][movie_name]
    
    # 그 사람들의 인덱스
    names = scores.index      

    # 그 사람들과의 상관관계
    corrs = data_corr[user_name].loc[names]
    
    # 예상점수 = 합( 유사도_v * 평점_v ) / 합(유사도_v)
    predict_dict[movie_name] = {'score' : np.sum(scores*corrs)/np.sum(corrs)}
    
    # print(predict_dict)
    
  predict_df = pd.DataFrame(predict_dict)
  
  return predict_df.T.sort_values(by='score', ascending=False)
  
movies = list(df.columns)
pred = predict_score('을지문덕', movies, df, df_corr)
pred

,score
사랑과전쟁,4.509960
세계대전,4.074777
넘버3,3.215408
겨울왕국,3.174858
택시운전사,2.248091
리빙라스베가스,2.193849


In [9]:
# 안본거 추천
def recommend(users, movies, df, df_corr ):
  
  rec_list = []
  for user_name in users:
    not_yet = df.T[df.T[user_name].isnull()][user_name].index # 안본 영화 리스트
    #print(not_yet)
    
    predict_df = predict_score(user_name, movies, df, df_corr) # 예측한 영화 순위 리스트
    
    
    if len(predict_df.loc[not_yet]) != 0:
      #print(rec_list)
      #print(user_name)
      rec_list.append((user_name, predict_df.loc[not_yet]))
      
  return rec_list
    
    
users = list(df.index)
recommend_result = recommend(users, movies, df, df_corr)

for obj, rec in recommend_result:
  print(obj, "에게 추천하는 영화")
  print(rec.sort_values(by='score', ascending = False))
  print('-----')


을지문덕 에게 추천하는 영화
            score
사랑과전쟁    4.509960
리빙라스베가스  2.193849
-----
김유신 에게 추천하는 영화
          score
택시운전사  2.606017
-----
이황 에게 추천하는 영화
            score
리빙라스베가스  2.475389
-----
이이 에게 추천하는 영화
            score
택시운전사    3.485804
세계대전     3.016121
리빙라스베가스  2.806620
-----


In [10]:
df

,겨울왕국,넘버3,리빙라스베가스,사랑과전쟁,세계대전,택시운전사
조용필,3.5,3.5,3.0,2.5,3.0,2.5
BTS,4.5,1.5,0.5,4.5,5.0,1.0
강감찬,3.5,5.0,1.5,3.5,3.0,3.0
을지문덕,3.0,3.5,NaN,NaN,4.0,2.5
김유신,3.5,4.0,3.0,2.5,4.5,NaN
유성룡,4.0,3.0,2.0,2.0,3.5,3.0
이황,4.0,5.0,NaN,3.5,3.0,3.0
이이,4.5,4.0,NaN,1.0,NaN,NaN
